In [ ]:
# use this notebook to check up on adversarial data

In [ ]:
import pandas as pd
from rlhfutils.debug_utils import load_all_hackdfs, highlight_differences
from datasets import load_dataset
import matplotlib.pyplot as plt

In [ ]:
appref = load_dataset("tatsu-lab/alpaca_farm", 'alpaca_human_preference')['preference']

In [ ]:
appref[10]

In [ ]:
train_dataset = load_dataset("csv", data_files="../rlcd-llama/simulated_data/simulated_preference_data_consolidated_helpful7b.csv")['train']


In [ ]:
def adjust_input(qval, rval):
    return "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n"+qval+"\n\n### Response:"+rval

def preproc_rlcd(example):
    ex = {}
    ex['question'] = example['instruction'][len("Human: "):-len("\n\nAssistant:")]
    if example['preference']==1:
        ex['response_j'] = adjust_input(ex['question'], example['output_1'])
        ex['response_k'] = adjust_input(ex['question'], example['output_2'])
    else:
        ex['response_k'] = adjust_input(ex['question'], example['output_1'])
        ex['response_j'] = adjust_input(ex['question'], example['output_2'])
    return ex

 
train_dataset = train_dataset.map(preproc_rlcd)

In [ ]:
train_dataset[0]

In [ ]:
train+

In [ ]:
rlcdpdset = pd.read_csv("../rlcd-llama/simulated_data/simulated_preference_data_consolidated_helpful7b.csv")

In [ ]:
rlcdpdset

In [ ]:
rlcdpdset['instruction'][0]

In [ ]:
def parse_row(examp):
    ex = {}
    bspl = examp['bestseqs'][-1].split("\n\nAnswer:")
    ospl = examp['origseq'].split("\n\nAnswer:")
    ex['question'] = bspl[0][len("Question: "):]
    ex['response_j'] = bspl[1]
    ex['response_k'] = ospl[1]
    return ex

In [ ]:
adfs = load_all_hackdfs("../rm-attacks/attackouts/dset/")

In [ ]:
dshuffs = pd.concat([adfs['dsetshuff'], adfs['dsetshuff2']]).reset_index(drop=True)

In [ ]:
imp = dshuffs[dshuffs['diff']>.05].reset_index(drop=True)


In [ ]:
imp.to_json("../outputs/augdata/actualdata.jsonl", orient='records', lines=True)

In [ ]:
adv_hacks = load_all_hackdfs("../rm-attacks/attackouts/reprodcheck/")
print(adv_hacks.keys())

In [ ]:
adv_hacks['advdatiebreak']['diff'].mean()

In [ ]:
def truncorig(odf, ndf):
    ndf = ndf[:150]
    odf['soseq'] = odf['origseq'].str[:150]
    ndf['soseq'] = ndf['origseq'].str[:150]
    # Drop duplicates to keep only the first instance of each unique 10-character substring
    odf.drop_duplicates(subset=['soseq'], inplace=True)
    ndf.drop_duplicates(subset=['soseq'], inplace=True)
    common_elements = set(list(odf['soseq'])).intersection(set(list(ndf['soseq'])))
    odf_filt = odf[odf['soseq'].isin(common_elements)].reset_index(drop=True)
    ndf_filt = ndf[ndf['soseq'].isin(common_elements)].reset_index(drop=True)
    odf_filt = odf_filt.sort_values(by='soseq')
    ndf_filt = ndf_filt.sort_values(by='soseq')
    return odf_filt, ndf_filt

In [ ]:
otrunc, ntrunc = truncorig(imp, adv_hacks['advdatiebreak'])

In [ ]:
print(ntrunc['diff'].mean())
print(len(ntrunc[ntrunc['diff']==0])/len(ntrunc))
plt.hist(ntrunc['diff'])

In [ ]:
ntrunc

In [ ]:
len(otrunc)

In [ ]:
otrunc['diff'].mean()

In [ ]:
ntrunc['diff'].mean()

In [ ]:
flip = nds.copy().rename(columns={'response_j':'response_k', 'response_k':'response_j'})

In [ ]:
flip

In [ ]:
fullaug = pd.concat([nds, flip]).reset_index(drop=True)

In [ ]:
fullaug

In [ ]:
fullaug.to_json('../outputs/augdata/'+"augstackv1.jsonl", lines=True, orient='records')

In [ ]:
read = pd.read_json('../outputs/augdata/'+"augstackv1.jsonl", lines=True, orient='records')

In [ ]:
read